# The Data Owner's Notebook


**Note:**

Much of the code used here is either copied or adapted from the `Word-level language modeling` PyTorch example:

https://github.com/pytorch/examples/tree/master/word_language_model

The goal being to demonstrate how the original example could be adapted to a context where the dataset is private to the data owner as it is the case in this demo.

## PART 0: Importing Libraries

In [1]:
import syft as sy

## PART 1: Launch Duet Server and Connect 

Let's start by launching the duet server. This server is launched by the data owner. The data scientist will connect to it afterwards.

In [2]:
duet = sy.launch_duet(loopback=True)

🎤  🎸  ♪♪♪ Starting Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently 
♫♫♫ > in alpha. Do not use this to protect real-world data.
♫♫♫ >
♫♫♫ > Punching through firewall to OpenGrid Network Node at: http://ec2-18-216-8-163.us-east-2.compute.amazonaws.com:5000
♫♫♫ > http://ec2-18-216-8-163.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... DONE!
♫♫♫ >
♫♫♫ > STEP 1: Send the following code to your Duet Partner!

import syft as sy
duet = sy.join_duet('e0f68000733933d11547491b578d2b7c')

♫♫♫ > STEP 2: The code above will print out a 'Client Id'. Have
♫♫♫ >         your duet partner send it to you and enter it below!

Running loopback mode. Use sy.join_duet(loopback=True) on the other side.
♫♫♫ > Connecting...
♫♫♫ > ...using a running event loop...

♫♫♫ > CONNECTED!



2020-11-07 18:11:29.751 | ERROR    | syft.grid.duet.webrtc_duet:push:201 - Got an exception in Duet push. 


♫♫♫ > DUET LIVE STATUS  *  Objects: 0  Requests: 0   Messages: 0                                

2020-11-07 18:11:32.015 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Launcher PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Launcher PQ: 0 / 0 - CQ: 0 / 0 - AT: 5
♫♫♫ > DUET LIVE STATUS  *  Objects: 0  Requests: 0   Messages: 0                                

...

Launcher PQ: 0 / 0 - CQ: 0 / 0 - AT: 5
♫♫♫ > DUET LIVE STATUS  -  Objects: 0  Requests: 0   Messages: 0                                

2020-11-07 18:11:42.020 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Launcher PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Launcher PQ: 0 / 0 - CQ: 0 / 0 - AT: 5
♫♫♫ > DUET LIVE STATUS  -  Objects: 0  Requests: 0   Messages: 0                                

2020-11-07 18:11:47.021 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Launcher PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Launcher PQ: 0 / 0 - CQ: 0 / 0 - AT: 5
♫♫♫ > DUET LIVE STATUS  *  Objects: 2  Requests: 0   Messages: 5                                Launcher PQ: 0 / 0 - CQ: 0 / 0 - AT: 5
♫♫♫ > DUET LIVE STATUS  *  Objects: 2  Requests: 0   Messages: 5                                Launcher PQ: 0 / 0 - CQ: 0 / 0 - AT: 5
♫♫♫ > DUET LIVE STATUS  -  Objects: 2  Requests: 0   Messages: 6                                Launcher PQ: 0 / 0 - CQ: 0 / 0 - AT: 5
♫♫♫ > DUET LIVE STATUS  -  Objects: 2  Requests: 0   Messages: 6                                Launcher PQ: 0 / 0 - CQ: 0 / 0 - AT: 5
♫♫♫ > DUET LIVE STATUS  -  Objects: 10  Requests: 0   Messages: 62                                Launcher PQ: 0 / 0 - CQ: 0 / 0 - AT: 5
♫♫♫ > DUET LIVE STATUS  *  Objects: 10  Requests: 0   Messages: 62                                Launcher PQ: 0 / 0 - CQ: 0 / 0 - AT: 5
♫♫♫ > DUET LIVE STATUS  *  Objects: 10  Requests: 0   Messages: 62                                Launcher PQ: 0 / 0 - CQ: 0 / 0 - AT: 5
♫♫♫ > DUET

## PART 2: Prepare Data

Let's create a class to hold the vocab of the dataset, with some utility methods.

In [3]:
import torch
import os

class Dictionary(object):
    """This class holds the vocabulary along
    with some utility functions.
    """
    def __init__(self):
        self.word2idx = {}
        self.idx2word = []

    def add_word(self, word):
        """Adds a word to the vocab.
        """
        
        if word not in self.word2idx:
            self.idx2word.append(word)
            self.word2idx[word] = len(self.idx2word) - 1
            
        return self.word2idx[word]

    def __len__(self):
        """Return the size of the used vocab
        """
        return len(self.idx2word)

Let's now create a class that preprocesses the dataset, and prepares it for both training and testing. In this particular use case, preprocessing includes tokenization and transforming words into integer IDs. 

In [4]:
class Corpus(object):
    
    def __init__(self, path):
        self.dictionary = Dictionary()
        self.train = self._tokenize(os.path.join(path, "train.txt"))
        self.valid = self._tokenize(os.path.join(path, "valid.txt"))
        self.test = self._tokenize(os.path.join(path, "test.txt"))

    def _tokenize(self, path):
        """Tokenizes a text file."""
        
        assert os.path.exists(path)
        
        # Add words to the dictionary
        with open(path, "r", encoding="utf8") as f:
            for line in f:
                words = line.split() + ["<eos>"]
                for word in words:
                    self.dictionary.add_word(word)

        # Tokenize file content
        with open(path, "r", encoding="utf8") as f:
            
            idss = []
            
            for line in f:
                
                words = line.split() + ["<eos>"]
                
                ids = []
                
                for word in words:
                    ids.append(self.dictionary.word2idx[word])
                    
                idss.append(torch.tensor(ids).type(torch.int64))
                
            ids = torch.cat(idss)

        return ids

Create a dataset instance for each of training, validation and testing, batchify, and share them on Grid:

In [5]:
# Create dataset
corpus = Corpus(path = 'data/')


Tag and describe the datasets before sharing on Duet.

In [6]:
# Training set
corpus.train.tag("wikitext2_dataset", 'train_data')
corpus.train.describe(f"Wikitext2 training set. shape: ({corpus.train.shape[0]},)")

# Validation set
corpus.valid.tag("wikitext2_dataset", 'valid_data')
corpus.valid.describe(f"Wikitext2 validation set. shape: ({corpus.valid.shape[0]},)")

# Test set
corpus.test.tag("wikitext2_dataset", 'test_data')
corpus.test.describe(f"Wikitext2 test set. shape: ({corpus.test.shape[0]},)")

tensor([   0,    1, 1144,  ...,   15,    0,    0])

Launcher PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Get the vocabulary size to share it on Duet:

In [7]:
vocab_size = sy.lib.python.Int(len(corpus.dictionary))
vocab_size.tag('wikitext2_dataset', 'vocab_size')
vocab_size.describe('Vocabulary size of Wikitext2 dataset')

2020-11-07 18:11:52.946 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Launcher PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


33278

## PART 3: Share Dataset on Duet

In [8]:
# Share the datasets on Grid
corpus.train.send(duet, searchable=True)

2020-11-07 18:12:05.723 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Launcher PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Launcher PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


In [ ]:
corpus.valid.send(duet, searchable=True)
#corpus.test.send(duet, searchable=True)

In [9]:
# Share the vocab size
vocab_size.send(duet, searchable = True)

2020-11-07 18:12:10.725 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Launcher PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Launcher PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Get a list of the shared objects:

In [10]:
duet.store.pandas

♫♫♫ > DUET LIVE STATUS  -  Objects: 2  Requests: 0   Messages: 3                                

,ID,Tags,Description
0,<UID:541fcbf4-8605-4af5-bfec-b2ba4a9db6f8>,"[wikitext2_dataset, vocab_size]",Vocabulary size of Wikitext2 dataset
1,<UID:d03deaaf-4ba8-4bce-a8f2-601cabcb2239>,"[wikitext2_dataset, train_data]","Wikitext2 training set. shape: (2088628,)"


2020-11-07 18:12:15.727 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Launcher PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Launcher PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-07 18:12:20.729 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Launcher PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


...

Launcher PQ: 0 / 0 - CQ: 0 / 0 - AT: 3


2020-11-07 19:14:22.424 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Launcher PQ: 0 / 0 - CQ: 0 / 0 - AT: 3


Launcher PQ: 0 / 0 - CQ: 0 / 0 - AT: 3


2020-11-07 19:14:27.425 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Launcher PQ: 0 / 0 - CQ: 0 / 0 - AT: 3


Launcher PQ: 0 / 0 - CQ: 0 / 0 - AT: 3


Automatically approve all requests for the sake of this demo

In [5]:
accept_handler = {
#     "request_name": "age_data",
    "timeout_secs": -1,
    "action": "accept",
    "print_local": True,
    "log_local": True
}
duet.requests.add_handler(accept_handler)

# while True:
#     if duet.requests:
#         for request in duet.requests:
#             request.approve()

2020-11-07 00:22:59.421 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Launcher PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Launcher PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


In [6]:
duet.requests.handlers

[{'timeout_secs': -1,
  'action': 'accept',
  'print_local': True,
  'log_local': True,
  'created_time': 1604701440.0}]

2020-11-07 00:23:04.424 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Launcher PQ: 0 / 0 - CQ: 0 / 0 - AT: 5
2020-11-07 00:23:06.181 | ERROR    | syft.core.node.common.action.get_object_action:execute_action:201 - Unable to Get Object with ID <UID:0a603c41-3305-4fa8-836b-7a7bfa5fdb54> from store. Possible dangling Pointer. <UID:0a603c41-3305-4fa8-836b-7a7bfa5fdb54>
2020-11-07 00:23:06.181 | ERROR    | syft.core.node.common.node:recv_immediate_msg_with_reply:336 - Unable to Get Object with ID <UID:0a603c41-3305-4fa8-836b-7a7bfa5fdb54> from store. Possible dangling Pointer. <UID:0a603c41-3305-4fa8-836b-7a7bfa5fdb54>
2020-11-07 00:23:06.182 | ERROR    | syft.core.node.common.node:recv_immediate_msg_with_reply:354 - An <class 'Exception'> has been triggered by <class 'syft.core.node.common.action.get_object_action.GetObjectAction'> from <Address - Domain:<SpecificLocation:..58e71>>
2020-11-07 00:23:09.425 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Launcher PQ: 0 

Launcher PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-07 00:23:15.389 | CRITICAL | syft.core.node.domain.service.request_message:send_msg:106 - Tried to Accept Message on Node. <UID:0a603c41-3305-4fa8-836b-7a7bfa5fdb54>
2020-11-07 00:23:15.395 | ERROR    | syft.core.node.common.action.get_object_action:execute_action:201 - Unable to Get Object with ID <UID:0a603c41-3305-4fa8-836b-7a7bfa5fdb54> from store. Possible dangling Pointer. <UID:0a603c41-3305-4fa8-836b-7a7bfa5fdb54>
2020-11-07 00:23:15.395 | CRITICAL | syft.core.node.domain.domain:check_handler:201 - Exception getting object. Unable to Get Object with ID <UID:0a603c41-3305-4fa8-836b-7a7bfa5fdb54> from store. Possible dangling Pointer. <UID:0a603c41-3305-4fa8-836b-7a7bfa5fdb54>
2020-11-07 00:23:15.402 | INFO     | syft.core.node.domain.domain:check_handler:216 - > Request :
Value: None
2020-11-07 00:23:19.433 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Launcher PQ: 0 / 0 - CQ: 0 / 0 - AT: 6
2020-11-07 00:23:20.537 | CRITICAL | syft.core.node.domain.service.req

<UID:0a603c41-3305-4fa8-836b-7a7bfa5fdb54>
> Request :
Value: None


...

Launcher PQ: 0 / 0 - CQ: 0 / 0 - AT: 3


2020-11-07 18:10:48.798 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Launcher PQ: 0 / 0 - CQ: 0 / 0 - AT: 3


Launcher PQ: 0 / 0 - CQ: 0 / 0 - AT: 3


2020-11-07 18:10:53.799 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Launcher PQ: 0 / 0 - CQ: 0 / 0 - AT: 3


Launcher PQ: 0 / 0 - CQ: 0 / 0 - AT: 3
